# Отладка

Напишем сначала функцию, которую будем отлаживать. Это будет функция, которая принимает на вход url сайта и ищет в нем заданную подстроку.

In [6]:
import requests
import re

def main(site_url, substring):
    site_code = get_site_code(site_url)
    matching_substrings = get_matching_substrings(site_code, substring)
    print(f'Substring {substring} is found {len(matching_substrings)} time(s) in {site_url}.')
    
def get_site_code(site_url):
    if not site_url.startswith('http'):
        site_url = 'http://' + site_url
    return requests.get(site_url).text

def get_matching_substrings(source, substring):
    return re.findall(substring, source)

main('mail.ru', 'mail')

Substring mail is found 993 time(s) in mail.ru.


За отладку отвечает модуль pdf, а включается он либо функцией `set_trace()`, либо через командную строку: `python -m pdf <file.py>`. Доступны в нем следующие команды:

**ОСНОВЫ**

- h(elp) - показать доступные команды.  
- h(elp) cmd - показать справку по команде cmd.  
- q(uit) - завершить работу программы.  

**РАБОТА С ДАННЫМИ**

- p(rint) expr - показать результат выражения expr.  
- pp expr - показать отформатированный результат выражения expr.  
- w(here) - показать текущий шаг и стек вызовов.  
- l(ist) - показать текущую строку кода и 11 строк вокруг нее.  
- l(ist) first last - показать строки с first до last.  
- a(rgs) - показать параметры, с которыми вызвана текущая функция.  

**ПОШАГОВОЕ ВЫПОЛНЕНИЕ**

- \<ENTER\> - повторно выполнить последнюю команду.  
- n(ext) - выполнить текущий шаг (step over).  
- s(tep) - провалиться внутрь текущего шага (step into).  
- r(eturn) - продолжить исполнение кода до завершения работы текущей функции.  
- c(ontinue) - продолжить исполнение кода до ближайшей точки останова.
- u(p) - перейти выше по стеку.  
- d(own) - перейти ниже по стеку.  

**ТОЧКИ ОСТАНОВА**

- b(reak) - показать все точки останова.  
- b(reak) n - поставить точку останова на строке n.  
- b(reak) func - поставить точку останова на первой строке функции func.  

**МАНИПУЛЯЦИИ**

- !stmt - рассматривать конструкцию stmt как выражение Python, а не как команду pdb.

Посмотрим это на примере нашей функции.

In [7]:
import requests
import re
import pdb

def main(site_url, substring):
    site_code = get_site_code(site_url)
    matching_substrings = get_matching_substrings(site_code, substring)
    print(f'Substring {substring} is found {len(matching_substrings)} time(s) in {site_url}.')
    
def get_site_code(site_url):
    if not site_url.startswith('http'):
        site_url = 'http://' + site_url
    return requests.get(site_url).text

def get_matching_substrings(source, substring):
    pdb.set_trace()
    return re.findall(substring, source)

main('mail.ru', 'mail')

> c:\users\kb255048\appdata\local\temp\ipykernel_2792\3494265520.py(17)get_matching_substrings()

ipdb> l
     12         site_url = 'http://' + site_url
     13     return requests.get(site_url).text
     14 
     15 def get_matching_substrings(source, substring):
     16     pdb.set_trace()
---> 17     return re.findall(substring, source)
     18 
     19 main('mail.ru', 'mail')

ipdb> w
    [... skipping 21 hidden frame(s)]

  c:\users\kb255048\appdata\local\temp\ipykernel_2792\3494265520.py(19)<module>()

  c:\users\kb255048\appdata\local\temp\ipykernel_2792\3494265520.py(7)main()

> c:\users\kb255048\appdata\local\temp\ipykernel_2792\3494265520.py(17)get_matching_substrings()

ipdb> q


BdbQuit: 

# Тестирование

Сначала простой пример. Это класс, который призван выполнять юнит-тесты. В данном случае тестируется стандартный функционал Python, однако в реальной разработке такие тестовые классы создаются для тестирования разработанного кода.

In [9]:
%%writefile test_python.py

import unittest

class TestPython(unittest.TestCase):
    def test_float_to_int_coerction(self):
        self.assertEqual(1, int(1.0))
        
    def test_get_empty_dict(self):
        self.assertIsNone({}.get('key'))
        
    def test_trueness(self):
        self.assertTrue(bool(10))

Writing test_python.py


Запустить тест из консоли можно следующим образом:

In [10]:
%%cmd

python -m unittest test_python.py

Microsoft Windows [Version 10.0.19043.1165]
(c) Microsoft Corporation. All rights reserved.

]9;8;"USERNAME"\@]9;8;"COMPUTERNAME"\ c:\Worker\Python\PythonMailRu\4.OOP-2
# ]9;12\
]9;8;"USERNAME"\@]9;8;"COMPUTERNAME"\ c:\Worker\Python\PythonMailRu\4.OOP-2
# ]9;12\python -m unittest test_python.py

]9;8;"USERNAME"\@]9;8;"COMPUTERNAME"\ c:\Worker\Python\PythonMailRu\4.OOP-2
# ]9;12\

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK


А теперь попробуем протестировать что-то более реалистичное. Это будет клиент, получающий данные по астероидам с сайта NASA.

In [12]:
%%writefile asteroid.py

import requests

class Asteroid:
    BASE_API_URL = 'https://api.nasa.gov/neo/rest/v1/neo/{}?api_key=DEMO_KEY'
    
    def __init__(self, spk_id):
        self.api_url = self.BASE_API_URL.format(spk_id)
    
    def get_data(self):
        return requests.get(self.api_url).json()
    
    @property
    def name(self):
        return self.get_data()['name']
    
    @property
    def diameter(self):
        return int(self.get_data()['estimated_diameter']['meters']['estimated_diameter_max'])
    
#apophis = Asteroid(2099942)
#
#print(f'Name: {apophis.name}')
#print(f'Diameter: {apophis.diameter}')

Writing asteroid.py


А теперь пишем тест...

In [18]:
%%writefile test_asteroid.py

import json
import unittest

from unittest.mock import patch
from asteroid import Asteroid

class TestAsteroid(unittest.TestCase):
    
    def setUp(self):
        self.asteroid = Asteroid(2099942)
        
    def mocked_get_data(self):
        with open('apophis_fixture.txt') as fd:
            return json.loads(fd.read())
        
    @patch('asteroid.Asteroid.get_data', mocked_get_data)
    def test_name(self):
        self.assertEqual(self.asteroid.name, '99942 Apophis (2004 MN4)')
        
    @patch('asteroid.Asteroid.get_data', mocked_get_data)
    def test_diameter(self):
        self.assertEqual(self.asteroid.diameter, 682)

Overwriting test_asteroid.py


In [19]:
%%cmd

python -m unittest test_asteroid.py

Microsoft Windows [Version 10.0.19043.1165]
(c) Microsoft Corporation. All rights reserved.

]9;8;"USERNAME"\@]9;8;"COMPUTERNAME"\ c:\Worker\Python\PythonMailRu\4.OOP-2
# ]9;12\
]9;8;"USERNAME"\@]9;8;"COMPUTERNAME"\ c:\Worker\Python\PythonMailRu\4.OOP-2
# ]9;12\python -m unittest test_asteroid.py
Name: 99942 Apophis (2004 MN4)
Diameter: 682

]9;8;"USERNAME"\@]9;8;"COMPUTERNAME"\ c:\Worker\Python\PythonMailRu\4.OOP-2
# ]9;12\

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK
